문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용 

 뉴스 키워드 추출기

문제 설명

FewShotPromptTemplate을 사용하여 뉴스 기사에서 핵심 키워드 3개를 추출하는 시스템을 구현해보세요. 주어진 예시들을 참고하여 일관된 형식으로 키워드를 추출해야 합니다.

요구사항

FewShotPromptTemplate 사용
최소 3개의 예시(examples) 포함
뉴스 텍스트에서 핵심 키워드 3개 추출
일관된 출력 형식 유지
다양한 분야의 뉴스로 테스트
예시 데이터


In [2]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [4]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="mistral-saba-24b",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x0000016A7FB8EAD0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000016A7FB8EE90> root_client=<openai.OpenAI object at 0x0000016A7FB8E850> root_async_client=<openai.AsyncOpenAI object at 0x0000016A7FB8EC10> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [12]:
from langchain_core.prompts import FewShotPromptTemplate,FewShotChatMessagePromptTemplate, PromptTemplate

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
         "news": "'3대 특검법' 국무회의 의결... 이재명 정부 1호 법안",
        "keywords": "3대 특검법, 국무회의, 이재명"
    }
    # 추가 예시 필요
]


# 2. Few-Shot 프롬프트
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 3. 최종 프롬프트
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
    few_shot_prompt,
    ("human", "{input}")
])



In [13]:
chain = final_prompt | llm

# 테스트 실행
result = chain.invoke({"input": "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."})
print(result.content)

키워드: 제미나이, 구글, AI
